### Test mapping

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mudata
import anndata
import scanpy as sc
import pandas as pd
import numpy as np

from typing import List, Dict, Union

In [36]:
import multi_view_atlas as mva

### Load dataset

In [4]:
import os
import multi_view_atlas as mva
datadir = '/nfs/team205/ed6/data/Fetal_immune/cellxgene_h5ad_files/scRNA_data/'
h5ad_files = [f for f in os.listdir(datadir) if f.endswith('embedding.h5ad')]

In [8]:
adata_full = sc.read_h5ad(os.path.join(datadir, 'PAN.A01.v01.raw_count.20210429.NKT.embedding.h5ad'), backed=False)

In [9]:
## Split in reference and query
query_cells = np.random.choice(adata_full.obs_names, size=int(np.round(adata_full.n_obs*0.1)), replace=False)
adata_query = adata_full[query_cells].copy()
adata_full = adata_full[~adata_full.obs_names.isin(query_cells)].copy()

In [13]:
assert adata_full.obs_names.isin(adata_query.obs_names).sum() == 0

In [142]:
### Split by organ
assign_dict = {
    'hematopoietic_tissue':['BM', "LI", "YS"],
    'lymphoid_tissue':['TH', "MLN", "SP"],
    'peripheral_tissue':['SK', "GU", 'KI']
}
annotation_col = 'organ'

assign_tab = np.vstack([np.where(adata_full.obs[annotation_col].isin(assign_dict[k]), 1, 0) for k in assign_dict.keys()]).T
assign_tab = pd.DataFrame(assign_tab, columns = assign_dict.keys(), index = adata_full.obs_names)

## Make dictionary of parent-child structure of views
view_hierarchy = { 'full':{
    'lymphoid_tissue':None,
    'hematopoietic_tissue':None,
    'peripheral_tissue':None
    }
}

In [141]:
import multi_view_atlas as mva

In [143]:
import traceback
adata_full.obsm['view_assign'] = assign_tab.copy()
adata_full.uns['view_hierarchy'] = view_hierarchy.copy()
# try:
mvatlas = mva.tl.MultiViewAtlas(adata_full, transition_rule='organ') 
# except:
#     v = traceback.format_exc()

/home/jovyan/my-conda-envs/patho-signatures-2/lib/python3.9/site-packages/mudata/_core/mudata.py:569: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


In [144]:
mvatlas.mdata.obs

,n_counts,n_genes,file,mito,doublet_scores,predicted_doublets,old_annotation_uniform,organ,Sort_id,age,method,donor,sex,Sample,scvi_clusters,is_maternal_contaminant,anno_lvl_2_final_clean,celltype_annotation
index,,,,,,,,,,,,,,,,,,
FCAImmP7579224-ATTATCCAGAGAACAG,39999.0,5076,FCAImmP7579224,0.038651,0.087221,False,nan,SK,CD45P,12,5GEX,F45,female,F45_SK_CD45P_FCAImmP7579224,14,False,LOW_Q_INCONSISTENT,LOW_Q_INCONSISTENT
FCAImmP7579224-GACGGCTAGCCACCTG,38114.0,5282,FCAImmP7579224,0.034633,0.110588,False,LTI/ILC3,SK,CD45P,12,5GEX,F45,female,F45_SK_CD45P_FCAImmP7579224,21,False,CYCLING_T,CYCLING_T
FCAImmP7579224-GCGGGTTGTCCGAGTC,33207.0,4690,FCAImmP7579224,0.028247,0.133690,False,SP T CELL,SK,CD45P,12,5GEX,F45,female,F45_SK_CD45P_FCAImmP7579224,21,False,CYCLING_T,CYCLING_T
FCAImmP7579224-AGTTGGTAGTGTTAGA,31058.0,4591,FCAImmP7579224,0.044433,0.133690,False,SP T CELL,SK,CD45P,12,5GEX,F45,female,F45_SK_CD45P_FCAImmP7579224,21,False,CYCLING_T,CYCLING_T
FCAImmP7579224-CGATCGGTCCCAAGTA,27622.0,4187,FCAImmP7579224,0.031678,0.077799,False,SP T CELL,SK,CD45P,12,5GEX,F45,female,F45_SK_CD45P_FCAImmP7579224,21,False,CYCLING_T,CYCLING_T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FCAImmP7277565-AGTAGTCCATGTAAGA,2028.0,892,FCAImmP7277565,0.006410,0.010212,False,DN T CELL,TH,CD45N,14,3GEX,F30,male,F30_TH_CD45N_FCAImmP7277565,2,False,DN(Q)_T,DN(Q)_T
FCAImmP7277565-GTCAAGTCATAAAGGT,2022.0,808,FCAImmP7277565,0.016320,0.026087,False,DN T CELL,TH,CD45N,14,3GEX,F30,male,F30_TH_CD45N_FCAImmP7277565,2,False,DN(Q)_T,DN(Q)_T
FCAImmP7277565-ATCATGGGTAAATGTG,2021.0,847,FCAImmP7277565,0.021771,0.028971,False,TREG,TH,CD45N,14,3GEX,F30,male,F30_TH_CD45N_FCAImmP7277565,11,False,TREG,TREG


### Update atlas

In [145]:
### Split by age in hematopoietic tissue
transition_rule = 'age'
parent_view = 'hematopoietic_tissue'

assign_tab = np.vstack([
    np.where(mvatlas.mdata[parent_view].obs[transition_rule] < 10, 1, 0),
    np.where(mvatlas.mdata[parent_view].obs[transition_rule] >= 10, 1, 0)
    ]).T
assign_tab = pd.DataFrame(
    assign_tab, 
    columns = ['first_trimester', 'second_trimester'], 
    index = mvatlas.mdata[parent_view].obs_names)


In [146]:
mvatlas.update_views(parent_view=parent_view, child_assign_tab=assign_tab, transition_rule=transition_rule)

In [133]:
np.fill_diagonal(mvatlas.view_transition_rule.values, np.nan)

In [156]:
get_views_from_structure(mvatlas.view_hierarchy)

['full',
 'lymphoid_tissue',
 'hematopoietic_tissue',
 'first_trimester',
 'second_trimester',
 'peripheral_tissue']

In [155]:
from multi_view_atlas.utils import *
child_v = 'T cells'
mvatlas.view_transition_rule.loc[get_parent_view(child_v, mvatlas.view_hierarchy), child_v] is not None

KeyError: None

In [147]:
mvatlas.view_transition_rule

,full,lymphoid_tissue,hematopoietic_tissue,peripheral_tissue,first_trimester,second_trimester
full,NaN,organ,NaN,organ,NaN,NaN
lymphoid_tissue,organ,NaN,NaN,NaN,NaN,NaN
hematopoietic_tissue,organ,NaN,NaN,NaN,age,age
peripheral_tissue,organ,NaN,NaN,NaN,NaN,NaN
first_trimester,NaN,NaN,age,NaN,NaN,NaN
second_trimester,NaN,NaN,age,NaN,NaN,NaN


In [75]:
mvatlas.mdata['full'][mvatlas.mdata.obsm["view_assign"]['first_trimester'] == 1]

View of AnnData object with n_obs × n_vars = 12943 × 0
    obs: 'n_counts', 'n_genes', 'file', 'mito', 'doublet_scores', 'predicted_doublets', 'old_annotation_uniform', 'organ', 'Sort_id', 'age', 'method', 'donor', 'sex', 'Sample', 'scvi_clusters', 'is_maternal_contaminant', 'anno_lvl_2_final_clean', 'celltype_annotation'
    obsm: 'X_scvi', 'X_umap', 'view_assign'
    obsp: 'scvi_connectivities', 'scvi_distances'

In [163]:
from multi_view_atlas.tl import MultiViewAtlas
adata = sample_dataset()
mva = MultiViewAtlas(adata)

/home/jovyan/my-conda-envs/patho-signatures-2/lib/python3.9/site-packages/mudata/_core/mudata.py:569: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


In [174]:
'CD4' in mva.view_transition_rule.columns

True

In [167]:
transition_rule = 'louvain'
parent_view = 'T cells'

assign_tab = np.vstack([
    np.where(mva.mdata[parent_view].obs[transition_rule] == 'CD4 T cells', 1, 0),
    np.where(mva.mdata[parent_view].obs[transition_rule] == 'CD8 T cells', 1, 0)
    ]).T
assign_tab = pd.DataFrame(
    assign_tab, 
    columns = ['CD4', 'CD8'], 
    index = mva.mdata[parent_view].obs_names)
mva.update_views(parent_view="T cells", child_assign_tab=assign_tab, transition_rule=transition_rule)
